In [2]:
sc.master

'local[*]'

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [68]:
path

'/Economics_submissions.txt'

In [4]:
# spark is from the previous example.
sc = spark.sparkContext

# A JSON dataset is pointed to by path.
# Move the 'people.json' file to hadoop /user1

path = "/Economics_submissions.txt"
#peopleDF = spark.read.json(path)

In [6]:
df = spark.read.text(path)

In [9]:
total_rows = df.count()

In [10]:
from pyspark.sql.functions import col

In [11]:
chunk_list = []

In [28]:
import json

In [36]:
def extract_title(json_str):
    try:
        json_obj = json.loads(json_str)
        title = json_obj.get('title', None)
        if title and isinstance(title,str):
            return title
    except Exception as e:
        pass
        return None

In [37]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [38]:
extract_title_udf = udf(extract_title, StringType())

In [39]:
df_titles = df.withColumn('title',extract_title_udf(col('value')))

In [40]:
df_valid_title = df_titles.filter(col('title').isNotNull())

In [42]:
df_valid_title.show()

+--------------------+--------------------+
|               value|               title|
+--------------------+--------------------+
|{"archived":true,...|The Best Economic...|
|{"archived":true,...|Meta: Seed The Ec...|
|{"archived":true,...|Meta: Seed The Ec...|
|{"archived":true,...|     Calculated Risk|
|{"archived":true,...|Chris Blattman's ...|
|{"archived":true,...|Consider the Evid...|
|{"archived":true,...|Dani Rodrik's weblog|
|{"archived":true,...|Freakonomics - Op...|
|{"archived":true,...|        gladwell.com|
|{"archived":true,...|Apple: Sandbaggin...|
|{"archived":true,...|When Iceland was ...|
|{"archived":true,...|My jaw just hit t...|
|{"archived":true,...|   Stimulus Gone Bad|
|{"archived":true,...|Stimulant or Hall...|
|{"archived":true,...|P.J. O'Rourke Tak...|
|{"archived":true,...|To fix economy, p...|
|{"archived":true,...|Update on Societe...|
|{"archived":true,...|Welfare for Wall ...|
|{"archived":true,...|Stimulus plan may...|
|{"archived":true,...|Delinquenc

In [89]:
def extract_date(json_str):
    try:
        json_obj = json.loads(json_str)
        utc = json_obj.get('created_utc', None)
        if utc and isinstance(utc,int):
            return utc
    except Exception as e:
        pass
        return None

In [92]:
extract_date_udf = udf(extract_date, IntegerType())

In [93]:
df_date = df.withColumn('created_utc',extract_date_udf(col('value')))

In [94]:
df_valid_date = df_date.filter(col('created_utc').isNotNull())

In [95]:
df_valid_date.show()

+--------------------+-----------+
|               value|created_utc|
+--------------------+-----------+
|{"archived":true,...| 1201236548|
|{"archived":true,...| 1201236599|
|{"archived":true,...| 1201236626|
|{"archived":true,...| 1201236662|
|{"archived":true,...| 1201236696|
|{"archived":true,...| 1201236717|
|{"archived":true,...| 1201236732|
|{"archived":true,...| 1201237559|
|{"archived":true,...| 1201237783|
|{"archived":true,...| 1201239067|
|{"archived":true,...| 1201270599|
|{"archived":true,...| 1201270753|
|{"archived":true,...| 1201271556|
|{"archived":true,...| 1201295011|
|{"archived":true,...| 1201312246|
|{"archived":true,...| 1201319383|
|{"archived":true,...| 1201323100|
|{"archived":true,...| 1201323505|
|{"archived":true,...| 1201323675|
|{"archived":true,...| 1201323782|
+--------------------+-----------+
only showing top 20 rows



In [96]:
selected_date = df_valid_date.select("created_utc")

In [97]:
selected_title = df_valid_title.select("title")

In [98]:
output_path = "/date_df_2.csv"

In [82]:
output_path_title = "/title_df_1.csv"

In [99]:
selected_date.write.csv(output_path,header=True, mode = 'overwrite')

In [84]:
selected_title.write.csv(output_path_title, header=True, mode = 'overwrite')